In [1340]:
import forallpeople
import handcalcs.render
from rjc.jupyter import title_block
import rjc.load_combinations as lc
forallpeople.environment('structural', top_level=True)

In [1341]:
# Inputs

proj = "UBC Gateway"
proj_id = "VAN.126897.0001"
des = "CMF"
member = "Orange Hi Load (c13-r2) @ l03"

# Program assumes that supports occur in the x-axis and y-axis concurrently
# i.e. L = L_x = L_y; L_u = L_u_x = L_u_y
# i.e. K_e = K_e_x = K_e_y

b = 215
d = 570

L = 4
L_u = 4
a = 4 # Purlin spacing for lateral support for bending
K_e = 1.0
n_plies = 2

Grade = "16c-E"
f_b_x = 14. # 30.6-24f-EX, 23.0-24f-E, 14.0-16c-E
f_b_y = 10.0 # f_b is taken as per No. 2 material from Table 6.3.1A
f_c = 32.0
E = 12800
K_T = 1
K_H = 1
K_S = 1
K_D = 0.88 # Check your DL and LL loads and confirm this applies

# For calculating K_D, area loads okay
P_f = 2024 # If P_f given, service loads ignored

e_x = 30#abs(d/2 - 418/2) #0.015 * d/2#abs(200 - d/2)  #d*0.05
e_y = 20 #0.015 * d/2#abs(200 - d/2)  #d*0.05

M_f_x = 0 # If M_f given, service loads ignored
M_f_y = 0 # If M_f given, service loads ignored_x


In [1342]:
# Inputs

b = b*mm
d = d*mm
L = L*m
L_u = L_u*m
a = a*m

f_b_x = f_b_x* MPa
f_b_y = f_b_y* MPa
f_c = f_c * MPa
E = E * MPa

# For calculating K_D, area loads okay
P_f = P_f * kN

e_x = e_x*mm
e_y = e_y*mm

M_f_x = M_f_x * kN*m
M_f_y = M_f_y * kN*m


In [1343]:
title_block(proj, proj_id, des, Member=member)

**Project:** UBC Gateway<br>
**Project ID:** VAN.126897.0001<br>
**Designer:** CMF<br>
**Date:** 2022-04-06<br>
**Member:** Orange Hi Load (c13-r2) @ l03<br>


# Compressive Resistance of Glulam Members (w/ Combined Bending)
As per CSA O86-14 Cl. 7

Assumes:

* $L = L_x = L_y$

# Loads

In [1344]:
%%render params
P_f

<IPython.core.display.Latex object>

In [1345]:
%%render
M_ecc_x = P_f * e_x
M_ecc_y = P_f * e_y

<IPython.core.display.Latex object>

In [1346]:
%%render
M_f_x_total = M_f_x + M_ecc_x
M_f_y_total = M_f_y + M_ecc_y

<IPython.core.display.Latex object>

# Section Properties

In [1347]:
%%render params
b
d
L
L_u
K_e
L_e = K_e * L 
Grade
f_b_x
f_b_y
f_c
E

<IPython.core.display.Latex object>

In [1348]:
%%render params
S_x = b * d**2 / 6
I_x = b * d**3 / 12
S_y = d * b**2 / 6
I_y = d * b**3 / 12

<IPython.core.display.Latex object>

### Modification Factors

In [1349]:
%%render params
K_T
K_H
K_S
K_D

<IPython.core.display.Latex object>

## Factored Stress Resistance

In [1350]:
%%render
F_c = f_c * (K_T * K_H * K_D * K_S)

<IPython.core.display.Latex object>

# Axial Resistance

## Slenderness Ratio, $C_c$

In [1351]:
%%render
C_c_x = L_e / d
C_c_y = L_e / b

<IPython.core.display.Latex object>

## Size Factor, $K_{Zcg}$

In [1352]:
%%render short
Z = (L * b * d).prefix('unity')
K_Zcg  = min(0.68*(float(Z))**-0.13, 1.0)

<IPython.core.display.Latex object>

## Slenderness Factor, $K_C$

In [1353]:
%%render
E_05 = 0.87 * E # Cl 7.5.8.5
K_C_x = (1.0 + (F_c * K_Zcg * C_c_x**3) / (35*E_05*K_S*K_T))**-1
K_C_y = (1.0 + (F_c * K_Zcg * C_c_y**3) / (35*E_05*K_S*K_T))**-1

<IPython.core.display.Latex object>

## Factored Compressive Resistance, $P_r$

In [1354]:
%%render
phi = 0.8
A = b * d

<IPython.core.display.Latex object>

In [1355]:
%%render long
P_r_x = phi * F_c * A * n_plies * K_Zcg * K_C_x
P_r_y = phi * F_c * A * n_plies * K_Zcg * K_C_y

<IPython.core.display.Latex object>

## Euler Buckling Load

In [1356]:
from math import pi

In [1357]:
%%render long
P_E_x = (pi**2 * E_05 * K_S * K_T * I_x) / L_e**2 * n_plies
P_E_y = (pi**2 * E_05 * K_S * K_T * I_y) / L_e**2 * n_plies

<IPython.core.display.Latex object>

## Secondary Moment Factor

In [1358]:
%%render
K_mo_2nd_x = (1 / (1 - P_f/P_E_x))
K_mo_2nd_y = (1 / (1 - P_f/P_E_y))

<IPython.core.display.Latex object>

# Bending Resistance

## Factored Stress Resistance

In [1359]:
%%render
F_b_x = f_b_x * (K_T * K_H * K_D * K_S)
F_b_y = f_b_y * (K_T * K_H * K_D * K_S)

<IPython.core.display.Latex object>

## Lateral Stability Factor, $K_L$

In [1360]:
from math import sqrt

In [1361]:
%%render
if (M_f_x == 0) and (M_f_y == 0): L_e = L_u # No applied lateral loads
else: L_e = 1.92*a
C_b_x = sqrt((L_e * d) / (b ** 2))
C_b_y = sqrt((L_e * b) / (d ** 2))
C_k_x = sqrt((0.97 * E * K_S * K_T) / F_b_x)
C_k_y = sqrt((0.97 * E * K_S * K_T) / F_b_y)

<IPython.core.display.Latex object>

In [1362]:
%%render
if C_b_x <= 10: K_L_x = 1.0
elif 10 <= C_b_x <= C_k_x: K_L_x = 1 - 1 / 3 * (C_b_x / C_k_x) ** 4
elif C_k_x <= C_b_x <= 50: K_L_x = (0.65 * E * K_S * K_T) / (C_b_x ** 2 * F_b_x)
elif C_b_x > 50: K_L_x = 0  # Cb is out of range

<IPython.core.display.Latex object>

In [1363]:
%%render
if C_b_y <= 10: K_L_y = 1.0
elif 10 <= C_b_y <= C_k_y: K_L_y = 1 - 1 / 3 * (C_b_y / C_k_y) ** 4
elif C_k_y <= C_b_y <= 50: K_L_y = (0.65 * E * K_S * K_T) / (C_b_y ** 2 * F_b_y)
elif C_b_y > 50: K_L_y = 0  # Cb is out of range

<IPython.core.display.Latex object>

## Size Factor, $K_{Zbg}$
Woodworks Sizer uses lamination width for $K_{Zbg}$. This calculation uses full member width.

In [1364]:
%%render
K_Zbg = (((130 * mm) / b) ** (1 / 10) * ((610 * mm) / d) ** (1 / 10) * ((9100 * mm) / L) ** (1 / 10))
K_Zbg = min(1.3, K_Zbg)

<IPython.core.display.Latex object>

# Moment Resistance, $M_{r1}$ and $M_{r2}$

### Strong axis (x)

In [1365]:
%%render
M_r1_x = phi * F_b_x * S_x * K_Zbg
M_r2_x = phi * F_b_x * S_x * K_L_x
M_r_x = min(M_r1_x, M_r2_x) * n_plies

<IPython.core.display.Latex object>

### Weak axis (y)

In [1366]:
%%render
M_r1_y = phi * F_b_y * S_y * K_Zbg
M_r2_y = phi * F_b_y * S_y * K_L_y
M_r_y = min(M_r1_y, M_r2_y) * n_plies

<IPython.core.display.Latex object>

## Combined Axial and Bending Resistance

### Strong axis (x)

In [1367]:
%%render
DCR_AXIAL_X = (P_f / P_r_x)
DCR_COMBINED_X = (P_f / P_r_x)**2 + M_f_x_total/M_r_x * K_mo_2nd_x

<IPython.core.display.Latex object>

### Weak axis (y)

In [1368]:
%%render
DCR_AXIAL_Y = (P_f / P_r_y)
DCR_COMBINED_Y = (P_f / P_r_y)**2 + M_f_y_total/M_r_x * K_mo_2nd_y

<IPython.core.display.Latex object>